This is the object detection code to detect the waste item in the environment

In [1]:
from ultralytics import YOLO
model=YOLO("C:\\Users\\Sriram\\Desktop\\Smart-Waste-AI\\model\\run\\detect\\garbage_detection_colab\\weights\\best.pt")
results=model("C:\\Users\\Sriram\\Desktop\\Smart-Waste-AI\\model\\glass.jpg", save=True)
results[0].show()


image 1/1 C:\Users\Sriram\Desktop\Smart-Waste-AI\model\glass.jpg: 640x640 1 glass, 141.8ms
Speed: 11.7ms preprocess, 141.8ms inference, 9.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\Sriram\runs\detect\predict9


This will show you the bounding box coordinates

In [2]:
for result in results:
  boxes=result.boxes
  print(boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([5.])
conf: tensor([0.2523])
data: tensor([[1.3301e+02, 2.6208e+01, 2.2551e+02, 3.2953e+02, 2.5231e-01, 5.0000e+00]])
id: None
is_track: False
orig_shape: (360, 360)
shape: torch.Size([1, 6])
xywh: tensor([[179.2649, 177.8682,  92.5000, 303.3207]])
xywhn: tensor([[0.4980, 0.4941, 0.2569, 0.8426]])
xyxy: tensor([[133.0149,  26.2079, 225.5148, 329.5286]])
xyxyn: tensor([[0.3695, 0.0728, 0.6264, 0.9154]])


This will give you the segmentation result of your detection model

In [ ]:
from ultralytics import YOLO, SAM

# Load YOLO model
yolo_model = YOLO("C:\\Users\\Sriram\\Desktop\\Smart-Waste-AI\\model\\run\\detect\\garbage_detection_colab\\weights\\best.pt")

# Run detection
results = yolo_model("C:\\Users\\Sriram\\Desktop\\Smart-Waste-AI\\model\\glass.jpg")

# Load SAM model - use 'cpu' as device
sam_model = SAM("sam_b.pt")  # Note: 'sam2_b.pt' might not be the correct name

for result in results:
    class_ids = result.boxes.cls.int().tolist()
    if len(class_ids):
        boxes = result.boxes.xyxy
        # Run SAM on CPU
        sam_results = sam_model(result.orig_img, bboxes=boxes, verbose=False, save=True, device='cpu')

This is for video segmentation

In [ ]:
from ultralytics import YOLO, SAM
import cv2

# Load YOLO model
yolo_model = YOLO("C:\\Users\\Sriram\\Desktop\\Smart-Waste-AI\\model\\run\\detect\\garbage_detection_colab\\weights\\best.pt")

# Load SAM model
sam_model = SAM("sam_b.pt")

# Input video path
video_path = "C:\\Users\\Sriram\\Desktop\\Smart-Waste-AI\\model\\test_video.mp4"
cap = cv2.VideoCapture(video_path)

# Output video path
output_path = "C:\\Users\\Sriram\\Desktop\\Smart-Waste-AI\\model\\video_segmentation_output(2).mp4"

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define VideoWriter
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    detect_results = yolo_model(frame)

    # Get bounding boxes
    boxes = detect_results[0].boxes.xyxy if detect_results[0].boxes else None

    if boxes is not None:
        # Run SAM segmentation on detected objects
        sam_results = sam_model(frame, bboxes=boxes, verbose=False)
        # Draw segmentation masks
        segmented_frame = sam_results[0].plot()
    else:
        segmented_frame = frame  # If no detections, keep original frame

    # Write the frame to output video
    out.write(segmented_frame)

# Release resources
cap.release()
out.release()
#cv2.destroyAllWindows()

print(f"Segmentation video saved to: {output_path}")